In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

: 

Mainly reading and preprocessing

In [6]:
dataframe=None

def sigmoid(z):
    z = np.array(z, dtype=float)
    return 1 / (1 + np.exp(-z))

def logisticRegression(X, y, theta, learning_rate, maxIteration, noFeatures):
    # print('hi')
    m=len(y)
    for iteration in range(maxIteration):
        # matrix multiplication
        h = sigmoid(np.dot(X,theta))
        gradient = np.dot(X.T,(h - y))
        theta = theta - learning_rate * gradient
    return theta

def predict(X, theta):
    probabilities = sigmoid(X @ theta)
    predictions = [1 if prob >= 0.5 else 0 for prob in probabilities]
    return np.array(predictions)

def scalingFunction(scaling='standard'):
    if scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'minmax':
        scaler = MinMaxScaler()
    
    return scaler

def normalize(X):
    # m,n=X.shape
    # for i in range(n):
    #     X=(X-X.mean(axis=0))/X.std(axis=0)
    return (X - X.mean(axis=0)) / X.std(axis=0)

def preprocessing(target_col_name):
    # drop null and fill null
    dataframe.dropna(subset=[target_col_name], inplace=True)
    dataframe.fillna(dataframe.mean(numeric_only=True),inplace=True)
    # fill null for non-numeric columns
    non_numerical_columns = dataframe.select_dtypes(include=['object']).columns
    for column in non_numerical_columns:
        mode_value = dataframe[column].mode()[0]
        dataframe[column].fillna(mode_value,inplace=True)
    # drop duplicates
    dataframe.drop_duplicates(inplace=True)
    # feature and target
    features=dataframe.drop(target_col_name,axis=1)
    target=dataframe[target_col_name]
    # label encoding the target
    encoder=LabelEncoder()
    target=encoder.fit_transform(target)
    # categorization and one-hot encoding
    categorical_columns=features.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        features[col]=features[col].astype('category')
    features=pd.get_dummies(features,columns=categorical_columns)
    # scaling
    candidate_columns=features.select_dtypes(exclude=['bool']).columns
    scaler=scalingFunction('standard')
    # scaler=scalingFunction('minmax')
    features_scaled=features.copy()
    features_scaled[candidate_columns]=scaler.fit_transform(features[candidate_columns])
    # transform to dataframe
    features_df=pd.DataFrame(features_scaled,columns=features.columns)
    target_df=pd.DataFrame(target,columns=[target_col_name])
    # adding for x0
    features_df.insert(0, 'x0', 1)
    # split into datasets
    X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    # model = LogisticRegression(max_iter=1000)
    # model.fit(X_train, y_train)
    # to numpy array
    X = X_train.to_numpy()
    X=normalize(X)
    y = y_train.to_numpy().flatten()
    maxIteration=1000
    noFeatures=features_df.shape[1]
    theta = np.random.randn(noFeatures,1)
    learning_rate = 0.01

    theta_final= logisticRegression(X, y, theta, learning_rate, maxIteration, noFeatures)
    X_test_np = X_test.to_numpy()
    predictions = predict(X_test_np, theta_final)
    y_test_np = y_test.to_numpy().reshape(-1)
    accuracy = np.mean(predictions == y_test_np) * 100
    print(accuracy)

def read_2():
    global dataframe
    column_file = 'adult/adult.names'
    data_file = 'adult/adult.data'
    columns = []
    with open(column_file, 'r') as f:
        for line in f:
            if '|' not in line:  # Ignore lines starting with '|'
                if ':' in line:
                    # Extract the column name before the ':' punctuation mark
                    col_name = line.split(':')[0].strip()
                    columns.append(col_name)
    columns.append('income-exceeds')
    dataframe = pd.read_csv(data_file, header=None)
    # print(dataframe)
    dataframe.columns = columns
    # print(dataframe)
    dataframe.replace(' ?', np.nan, inplace=True)

def read(input):
    global dataframe
    # file 1
    if input == 1:
        dataframe = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
        preprocessing('Churn')
    # file 2
    elif input == 2:
        read_2()
        preprocessing('income-exceeds')
    # file 3
    elif input == 3:
        dataframe = pd.read_csv('creditcard.csv')
        preprocessing('Class')
    else:
        print("Invalid input")
    # dataframe.head()

user_input = int(input("Enter 1, 2 or 3: "))
read(user_input)

C:\Users\hp\AppData\Local\Temp\ipykernel_8572\2698451137.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe[column].fillna(mode_value,inplace=True)


In [6]:
# rows,cols=dataframe.shape
# cols,rows
# dataframe.describe()
# dataframe.isnull().sum()
# dataframe.duplicated().sum()
# Data cleaning
# dataframe['Attrition'].isnull().sum()
# # drop the rows where the target value is null
# dataframe.dropna(subset=['Attrition'], inplace=True)
# dataframe['Attrition'].isnull().sum()
# dataframe.isnull().sum()
# dataframe.shape
# dataframe.fillna(dataframe.mean(numeric_only=True),inplace=True)
# non_numerical_columns = dataframe.select_dtypes(exclude=['int64', 'float64']).columns
# for column in non_numerical_columns:
#     mode_value = dataframe[column].mode()[0]
#     dataframe[column].fillna(mode_value,inplace=True)
# dataframe.isnull().sum()
# dataframe.duplicated().sum()
# dataframe.drop_duplicates(inplace=True)
# dataframe.duplicated().sum()
# # Creation of input and output features
# features=dataframe.drop('Attrition',axis=1)
# target=dataframe['Attrition']
# #Conversion of features into numeric values
# from sklearn.preprocessing import LabelEncoder
# encoder=LabelEncoder()
# target=encoder.fit_transform(target)
# target
# categorical_columns=features.select_dtypes(include=['object']).columns
# for col in categorical_columns:
#     features[col]=features[col].astype('category')
# features=pd.get_dummies(features,columns=categorical_columns)
# features
# features.dtypes
# Scaling of the features
# candidate_columns=features.select_dtypes(exclude=['bool']).columns
# candidate_columns
# from sklearn.preprocessing import StandardScaler,MinMaxScaler
# def scalingFunction(scaling='standard'):
#     if scaling == 'standard':
#         scaler = StandardScaler()
#     elif scaling == 'minmax':
#         scaler = MinMaxScaler()
    
#     return scaler

# scaler=scalingFunction('standard')
# # scaler=scalingFunction('minmax')
# features_scaled=features.copy()
# features_scaled[candidate_columns]=scaler.fit_transform(features[candidate_columns])
# features_scaled

#  Correlation Analysis
# # transform from numpy array to dataframe
# features_df=pd.DataFrame(features_scaled,columns=features.columns)
# target_df=pd.DataFrame(target,columns=['Attrition'])
# target_df
# target_series=target_df['Attrition']
# correlation=features_df.corrwith(target_series)
# correlation
# from matplotlib import pyplot as plt
# import seaborn as sns
# features_with_target = pd.concat([features_df, target_series], axis=1)
# correlation_matrix = features_with_target.corr()
# plt.figure(figsize=(30, 30))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.1f')
# plt.title('Correlation Matrix')
# plt.show()
# import numpy as np

# features_df.reset_index(drop=True, inplace=True)
# target_df.reset_index(drop=True, inplace=True)

# class_0=features_df.loc[target_df['Attrition']==0]
# class_1=features_df.loc[target_df['Attrition']==1]

# def plot(column):
#     plt.plot(class_0[column],np.zeros_like(class_0[column]),'o',label='No')
#     plt.plot(class_1[column],np.zeros_like(class_1[column]),'x',label='Yes')

#     plt.legend()
#     plt.xlabel(column)
#     plt.title(f"1D scatter plot of {column}")
#     plt.show()

# top_20_features = correlation.abs().nlargest(20).index
# print(top_20_features)
# for feature in top_20_features:
#     plot(feature)

# Validating the pipeline (Bonus Task)
# top_20_features = correlation.index.tolist() #standard scaling
# features_df = features_df[top_20_features]
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score

# X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size=0.2, random_state=42)

# model = LogisticRegression()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# accuracy